In [6]:
!pip3 install SpeechRecognition

#from google.colab import drive
#drive.mount ("/content/drive/")
#path="drive/MyDrive/MultiMed/"

path="assets/"

import speech_recognition as sr
import textwrap
import nltk
nltk.download("punkt")
from nltk import word_tokenize, sent_tokenize
import numpy as np

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
!pip3 install pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [10]:
# Import long audio file
filename = "alices-adventures-in-wonderland-001-chapter-i-down-the-rabbit-hole.1.mp3"
audio_data = AudioSegment.from_file(path+filename)

In [11]:
# Split audio into chunks based on pauses
chunks = split_on_silence(audio_data,1000,audio_data.dBFS-20,100)

# Transcribe audio chunks into text chunks and save
i = 0
for chnk in chunks:
  filename = "chunk_"+str(i)+".wav"
  chnk.export(path+filename,format="wav")
  r = sr.Recognizer()
  with sr.AudioFile(path+filename) as source:
    audio = r.record(source)
    text = r.recognize_google(audio)
    filename = "chunk_"+str(i)+".txt"
    f = open(path+filename, "w")
    f.write(text)
    f.close
  i += 1

22


In [122]:
# Read files and create a case-insensitive BoW
i = 0
w_tokens = np.empty(1,str)
for chnk in chunks:
  filename = "chunk_"+str(i)+".txt"
  f = open(path+filename, "r")
  rawtext = f.read()
  rawtext = rawtext.lower()
  f.close()
  w_tokens = np.append(w_tokens,word_tokenize(rawtext))
  i += 1

In [123]:
# Remove: puncuation, stopwords
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
abbv = ["n't", "'s", "'re", "'m", "na", "ca", "gon", "'ll"]
stpwd = stopwords.words("english")

matched_indices = []

for tok in range(0,len(w_tokens)):
  if re.search(r"\w",w_tokens[tok]) and w_tokens[tok] not in stpwd and w_tokens[tok] not in abbv:
    matched_indices.append(tok)

w_tokens2 = []

for i in matched_indices:
  w_tokens2.append(w_tokens[i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
# Remove: Words appearing less than 5 times
unique,indices,counts = np.unique(w_tokens2, return_index=True, return_counts=True)
unique = unique[np.flip(np.argsort(counts))]
counts = counts[np.flip(np.argsort(counts))]

# Create dict of final tokens
huff_tokens = {}
for key, val in zip(unique, counts):
  if val > 4:
    huff_tokens[key] = val
print(huff_tokens)

{'alice': 25, 'little': 15, 'way': 14, 'see': 11, 'think': 11, 'like': 11, 'could': 9, 'rabbit': 9, 'time': 9, 'one': 9, 'said': 8, 'would': 8, 'found': 7, 'door': 7, 'get': 7, 'nothing': 7, 'say': 6, 'thought': 6, 'wonder': 6, 'much': 6, 'going': 6, 'went': 6, 'know': 5, 'eat': 5, 'never': 5, 'tried': 5, 'either': 5, 'things': 5, 'suddenly': 5, 'table': 5, 'key': 5, 'use': 5, 'shall': 5}


In [125]:
# Huffman coding: code referenced from https://www.geeksforgeeks.org/huffman-coding-greedy-algo-3/

import heapq
# Define node object
class node:
	def __init__(self, count, token, left=None, right=None):
		self.count = count
		self.token = token
		self.left = left
		self.right = right
		self.code = ''

	def __lt__(self, nxt):
		return self.count < nxt.count

# Create nodes and add to heap
nodes = []
for tok in huff_tokens.items():
	heapq.heappush(nodes, node(tok[1], tok[0]))

# While still unprocessed nodes...
while len(nodes) > 1:
  # Sort in ascending order
  left = heapq.heappop(nodes)
  right = heapq.heappop(nodes)
  # Assign directional value
  left.code = 0
  right.code = 1
  # Create parent node and push to heap
  newNode = node(left.count+right.count, left.token+right.token, left, right)
  heapq.heappush(nodes, newNode)

In [126]:
def encodeNodes(node, val=''):
	# Generate Huffman code for current node
  newVal = val + str(node.code)
	# if node is not an edge node, recurse
  if(node.left):
    encodeNodes(node.left, newVal)
  if(node.right):
    encodeNodes(node.right, newVal)
	# if node is edge node, add node with code to array
  if(not node.left and not node.right):
    node.code = newVal
    nodes.append(node)

# Generate Huffman codes for tree
encodeNodes(nodes[0])
# Remove root from list of nodes
nodes.pop(0)

In [127]:
# Display codes in descending order
nodes.sort(key=lambda n: n.count, reverse=True)
print(f'{"Word":15}{"Code":15}{"Count"}')
print("----------------------------------")
for n in nodes:
  print(f'{n.token:15}{n.code:15}{n.count}')

Word           Code           Count
----------------------------------
alice          001            25
little         1000           15
way            0110           14
think          11100          11
see            11101          11
like           11111          11
could          10011          9
one            10100          9
rabbit         10101          9
time           10110          9
said           01111          8
would          10010          8
get            01001          7
door           01010          7
found          01011          7
nothing        01110          7
much           00000          6
went           00001          6
going          00010          6
say            00011          6
wonder         01000          6
thought        111101         6
know           101110         5
eat            101111         5
things         110000         5
table          110001         5
either         110010         5
use            110011         5
key            110100      

In [142]:
# Ask for a code, search for word and display
import string
srch = input("Enter the code/bits to select a word: ")
for n in nodes:
  if n.code == srch:
    word = " "+n.token+" "

print("\n\"",word,"\" is found in the following text: \n")
i = 0
for chnk in chunks:
  filename = "chunk_"+str(i)+".txt"
  f = open(path+filename, "r")
  rawtext = " "+f.read()+" "
  if word.casefold() in rawtext.casefold():
    print("text",str(i),": ",rawtext)
  f.close()
  i += 1

Enter the code/bits to select a word: 110011

"  use  " is found in the following text: 

text 1 :   Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures of conversation so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her there was nothing so very remarkable in that nor did Alice think it's so very much out of the way to hear the rabbit say to itself oh dear oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit a